In [2]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import curio
import curio_http

ROOT = Path('../')
DATA_DIR = ROOT/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

from countdowner import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Test some
w = read_watchlist(DATA_DIR/'test_watchlist.yaml')
w


{'email_address': 'brainbummer@mailinator.com',
 'name': 'test',
 'products':                          description stock_code
 0                     organic cheese     281739
 1                       GB chocolate     260803
 2                     Lupi olive oil     701829
 3      Earthcare double toilet paper     381895
 4                     Dijon mustard      700630
 5                whole grain mustard     700631
 6            Pics peanut butter 380g     360257
 7           Pics peanut butter 1000g     887052
 8                      coconut cream     271818
 9     Ecostore dishwash liquid 500mL     701108
 10   Ecostore dishwash liquid 1000mL     797559
 11   Earthwise dishwash liquid 500mL     701227
 12  Earthwise dishwash liquid 1000mL     504018}

In [4]:
codes = w['products']['stock_code']
%time g = curio.run(collect_products_a(codes))
g

CPU times: user 936 ms, sys: 12 ms, total: 948 ms
Wall time: 1.75 s


,stock_code,name,description,size,on_sale,sale_price,price,discount,unit_price,datetime
8,271818,Ceres Organics Coconut Cream Creamy & Unsweete...,Creamy and unsweetened,can 400ml,False,NaN,4.25,NaN,$1.06/100G,2017-05-29 13:27:04
3,381895,Earthcare Toilet Paper 6pk Double Lenght Sky ...,None,6pk,False,NaN,5.00,NaN,$0.19/100SS,2017-05-29 13:27:03
11,701227,Earthwise Dishwash Liquid 500ml,None,500ml,False,NaN,3.99,NaN,$0.80/100ML,2017-05-29 13:27:04
12,504018,Earthwise Dishwash Liquid Pink Grapefruit 1l,None,1l,False,NaN,7.00,NaN,$0.70/100ML,2017-05-29 13:27:04
10,797559,Ecostore Dishwash Liquid Grapefruit 1l,Powerful formulation for sparkling clean dishe...,1l,False,NaN,7.39,NaN,$0.74/100ML,2017-05-29 13:27:04
9,701108,Ecostore Dishwash Liquid Grapefruit 500ml,Powerful formulation for sparkling clean dishe...,500ml,True,3.19,3.95,0.192405,$0.64/100ML,2017-05-29 13:27:04
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,True,3.50,3.89,0.100257,$3.50/100G,2017-05-29 13:27:03
2,701829,Lupi Olive Oil Organic Extra Virgin 750ml,None,750ml,False,NaN,15.00,NaN,$2.00/100ML,2017-05-29 13:27:03
4,700630,Macro Organic Mustard Dijon 200g,None,200g,True,2.99,3.49,0.143266,None,2017-05-29 13:27:03
5,700631,Macro Organic Mustard Wholegrain 200g,None,200g,True,2.99,3.49,0.143266,None,2017-05-29 13:27:04


In [5]:
filter_sales(g)


,name,sale_price,price,discount
9,Ecostore Dishwash Liquid Grapefruit 500ml,3.19,3.95,0.192405
1,Green & Blacks Chocolate Block Organic Dark Ch...,3.50,3.89,0.100257
4,Macro Organic Mustard Dijon 200g,2.99,3.49,0.143266
5,Macro Organic Mustard Wholegrain 200g,2.99,3.49,0.143266
7,Pics Peanut Butter Crunchy 1kg,16.50,18.49,0.107626


In [9]:
%time run_pipeline(DATA_DIR/'watchlist_test.yaml', OUT_DIR, key=get_secret('mailgun_api_key'), as_html=True)

CPU times: user 984 ms, sys: 12 ms, total: 996 ms
Wall time: 7.47 s
